# Imports

Clone and install https://github.com/MTG/pycompmusic

Download data using `download-dunya.py`

In [ ]:
import json, os, sys
import pickle, csv
import time
import datetime
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px

import scipy
import librosa
from IPython.display import Audio as ipy_audio

# Data

In [ ]:
data_dir = "/Users/neerajaabhyankar/Repos/icm-shruti-analysis/data-dunya-hindustani/"

In [ ]:
track = "Omkar Dadarkar - Raag Bhoopali"

In [ ]:
audio_file = track + ".mp3"
metadata_file = track + ".json"
tonic_file = track + ".ctonic.txt"
pitch_file = track + ".pitch.txt"

Metadata + Tonic

In [ ]:
with open(data_dir + metadata_file) as json_data:
    metadata = json.load(json_data)

tonic = float(np.loadtxt(data_dir + tonic_file))

In [ ]:
print(f"length = {metadata['length']/1000} seconds")
print(f"base tonic = {tonic}")

Audio File

In [ ]:
# y, sr = librosa.load(data_dir + audio_file)
# ipy_audio(data=y, rate=sr)

Spectrogram

In [ ]:
# D = librosa.stft(y)
# S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

# plt.figure().set_figwidth(12)
# librosa.display.specshow(S_db, x_axis="time", y_axis="hz", bins_per_octave=22)
# plt.colorbar()

Pitch Annotations

In [ ]:
pitch_annotations = []
with open(data_dir + pitch_file) as pf:
    for line in pf:
        pitch_annotations.append(line.split())
pitch_annotations = np.array(pitch_annotations).astype(float)

In [ ]:
aps = int(pitch_annotations.shape[0]/metadata['length']*1000)
print(f"{aps} annotations per second")

# Validating Pitch Annotations

Take a representative sample

In [ ]:
# 06:45 to 07:00
start = 6*60+45
end = 7*60

y_small = y[start*sr:end*sr]
pa_small = pitch_annotations[start*aps:end*aps, 1]

In [ ]:
# downsample so as to make clearer tones
downsample_factor = 25
pa_small_d = pa_small[::downsample_factor]

In [ ]:
# create a waveform
y_pa_small = []
for anno in pa_small_d:
    tone = librosa.tone(2*anno, sr=sr, length=downsample_factor*sr/aps)
    y_pa_small += tone.tolist()

print("annotaion-generated sample")
ipy_audio(data=y_pa_small, rate=sr)

In [ ]:
print("original sample")
ipy_audio(data=y_small, rate=sr)

# TODO

NEXT:
1. refine the beat structure to 1/4
2. auto-detect tempo chunks, break by tempo, and apply the prototype
3. try some basic librosa pitch detection, see if it's better than the annotations, else just keep taking the mode over annotations
3. now we have a real time-series in φ

Select a sample

In [ ]:
# 06:45 to 07:00
start = 45*60+20
end = 45*60+32

y_small = y[start*sr:end*sr]
pa_small = pitch_annotations[start*aps:end*aps, 1]

In [ ]:
tempo, beat_frames = librosa.beat.beat_track(y=y_small, sr=sr)
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

Temp: collapse to known swars

In [ ]:
swar = {
    "D.": 0.8362, "S": 0.997, "R": 1.11896, "G": 1.2565, "P": 1.501
}
swars = np.array(list(swar.values()))*tonic
swars = np.concatenate((swars/2, swars, swars*2))

In [ ]:
swar_small = []
for pa in pa_small:
    if pa == 0:
        swar_small.append(0)
    else:
        idx = (np.abs(swars - pa)).argmin()
        swar_small.append(swars[idx])

# plt.plot(range(len(pa_small)), pa_small)
# plt.plot(range(len(swar_small)), swar_small)

Note: the following won't scale! Just for now, looking at how the mode sounds

In [ ]:
beat_swars = []
y_swar_small = []
for beat_idx in range(1, len(beat_times)):
    beat_start = beat_times[beat_idx-1]
    beat_end = beat_times[beat_idx]
    swars_beat = swar_small[int(beat_start*aps):int(beat_end*aps)]
    
    idx = (np.abs(swars - scipy.stats.mode(swars_beat).mode)).argmin()
    beat_swars.append(swars[idx])
    
    tone = librosa.tone(swars[idx], sr=sr, length=(beat_end-beat_start)*sr)
    y_swar_small += tone.tolist()

    

In [ ]:
print("beat-wise swar-collapsed waveform")
ipy_audio(data=y_swar_small, rate=sr)

In [ ]:
print("original sample")
ipy_audio(data=y_small, rate=sr)

# Manual Note Mode Finding

Pitch Histogram

In [ ]:
nz_annotations = pitch_annotations[np.where(pitch_annotations[:,1] != 0), 1][0]
h = plt.hist(nz_annotations/tonic, bins=700)
plt.xscale("log")
_ = plt.xlim(0.5, 3)

counts = h[0]
notes = (h[1][:-1]+h[1][1:])/2

In [ ]:
fig = px.line(
    pd.DataFrame({"note": notes, "count": counts}),
    x="note", y="count",
    log_x=True
)
# fig.update_layout(xaxis_range=[0.5, 3.1])
fig.update_traces(hovertemplate="freq_ratio: %{x}<br>occurence: %{y}")
fig.show()

## TODO: Bunch nearby frequencies and do this automatically

# Manual Peak Finding Results

Add counts to the list Below

In [ ]:
raag = ""  # Omkar Dadarkar
swar = {
#     "D.": 0.8362,
#     "S": 0.997,
#     "R": 1.11896,
#     "G": 1.2565,
#     "P": 1.501,
#     "D": 1.6843,  # may be lower
#     "S^": 1.998,
}

In [ ]:
raag = "Bhoopali"  # Omkar Dadarkar
swar = {
    "D.": 0.8362,
    "S": 0.997,
    "R": 1.11896,
    "G": 1.2565,
    "P": 1.501,
    "D": 1.6843,  # may be lower
    "S^": 1.998,
}

In [ ]:
raag = "Multani"  # Omkar Dadarkar
swar = {
    "N.": 0.942,
    "S": 1.002,
    "r": 1.050,
    "g": 1.1923,
    "m": 1.416,  # may be higher
    "P": 1.497,
    "d": 0.0,  #?
    "N": 1.8896,
    "S^": 1.998,
}

In [ ]:
raag = "Todi"  # Omkar Dadarkar
swar = {
    "P.": 0.7495,
    "d.": 0.7984,  #?
    "S": 1.000,  # originally was 0.992
    "r": 1.0615,
    "g": 1.1895,
    "m": 1.434,
    "P": 1.508,  # may be higher
    "d": 1.594,
    "N": 1.9083,
    "S^": 2.001,
    "r^": 2.389,
}
# all OG values divivded by 0.992

In [ ]:
raag = "Bhimpalasi"  # Satyasheel Deshpande
swar = {
    "n.": 0.901,
    "S": 1.000,
    "R": 1.127,
    "g": 1.197,  # may be lower
    "M": 1.333,
    "P": 1.502,  # may be lower
    "D": 1.678,  # may be lower
    "n": 1.791,  # may be lower
    "S^": 2.00,
}

In [ ]:
raag = "Bhimpalasi"  # Omkar Dadarkar
swar = {
    "n.": 0.888, 
    "S": 1.002,
    "R": 1.1225,
    "g": 1.189,  # may be higher
    "M": 1.332,
    "P": 1.495,  # may be higher
    "D": 1.689,  # may be lower
    "n": 1.784,  # may be lower
    "S^": 2.00,
    "R^": 2.37,
}